*****GENERATING INDICES DATA*****

In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
from tabulate import tabulate

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    base_path = Path("C:/Users/manoj/Downloads/Major project data/Major pro source codes/DATASETS")
    daylocationstr = base_path / "Daily_data" / f"{symbol}.csv"
    weeklocationstr = base_path / "Weekly_data" / f"{symbol}.csv"
    monthlocationstr = base_path / "Monthly_data" / f"{symbol}.csv"

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname.strip(),
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    base_path = Path("C:/Users/manoj/Downloads/Major project data/Major pro source codes/DATASETS")
    fname = base_path / scripttype
    csvfilename = base_path / f"GFS_{scripttype}.csv"

    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Generate the GFS report
df3 = generateGFS("indicesdf.csv")

# Display the DataFrame in tabular format using tabulate
print(tabulate(df3, headers='keys', tablefmt='fancy_grid'))


*****GFS LOGIC FOR INDICES*****

In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
from tabulate import tabulate
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame.

    Args:
        df (pd.DataFrame): The existing DataFrame.
        row (pd.Series): The new row to append.

    Returns:
        pd.DataFrame: The DataFrame with the new row appended.
    """
    return pd.concat([
        df,
        pd.DataFrame([row], columns=row.index)
    ]).reset_index(drop=True)

def getRSI14_and_BB(csvfilename):
    """Calculates RSI (14 period) and Bollinger Bands (20 period, 2 std.dev) for a given CSV file.

    Args:
        csvfilename (str): Path to the CSV file.

    Returns:
        tuple: A tuple containing RSI value, last close price, lower band, and middle band.
        (float, float, float, float)
    """
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty or 'Close' not in df.columns:
                return 0.00, 0.00, 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                bb = ta.bbands(df['Close'], length=20)
                if bb is None or df['rsi14'] is None:
                    return 0.00, 0.00, 0.00, 0.00
                df['lowerband'] = bb['BBL_20_2.0']
                df['middleband'] = bb['BBM_20_2.0']
                if pd.isna(df['rsi14'].iloc[-1]) or pd.isna(df['lowerband'].iloc[-1]) or pd.isna(df['middleband'].iloc[-1]):
                    return 0.00, 0.00, 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                    lowerband = df['lowerband'].iloc[-1].round(2)
                    middleband = df['middleband'].iloc[-1].round(2)
                    return rsival, ltp, lowerband, middleband
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00, 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00, 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI, Bollinger Bands, and other metrics for daily, weekly, and monthly data.

    Args:
        symbol (str): The symbol of the asset.
        symbolname (str): The name of the asset.

    Returns:
        pd.Series: A Series containing the calculated metrics.
    """
    daylocationstr = f'DATASETS/Daily_data/{symbol}.csv'
    weeklocationstr = f'DATASETS/Weekly_data/{symbol}.csv'
    monthlocationstr = f'DATASETS/Monthly_data/{symbol}.csv'

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp, daylowerband, daymiddleband = getRSI14_and_BB(daylocationstr)
    weekrsi14, wltp, weeklowerband, weekmiddleband = getRSI14_and_BB(weeklocationstr)
    monthrsi14, mltp, monthlowerband, monthmiddleband = getRSI14_and_BB(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14,
        'dltp': dltp,
        'daylowerband': daylowerband,
        'daymiddleband': daymiddleband,
        'weeklowerband': weeklowerband,
        'weekmiddleband': weekmiddleband,
        'monthlowerband': monthlowerband,
        'monthmiddleband': monthmiddleband
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype.

    Args:
        scripttype (str): The name of the scripttype file.

    Returns:
        pd.DataFrame: The DataFrame containing the GFS report.
    """
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14', 'dltp', 'daylowerband', 'daymiddleband', 'weeklowerband', 'weekmiddleband', 'monthlowerband', 'monthmiddleband'])

    fname = f'DATASETS/{scripttype}.csv'
    csvfilename = f'GFS_{scripttype}.csv'
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.replace("\n", "")
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Assuming 'DATASETS/indicesdf.csv' exists and is in the correct format
df3 = generateGFS('indicesdf')

df4 = df3.loc[
    (df3['monthrsi14'] >= 60.00) &
    (df3['weekrsi14'] >= 60.00) &
    df3['dayrsi14'].between(30, 70) &
    (df3['dltp'] > df3['daylowerband']) &
    (df3['dltp'] < df3['daymiddleband'])
]

df4 = df4.sort_values(by=['dayrsi14'], ascending=True)

if df4.empty:
    print("\033[1mNO INDICES QUALIFIES THE GFS CRITERIA\033[0m")
else:
    print(tabulate(df4, headers='keys', tablefmt='fancy_grid', showindex=False))

*****STOCKS DATA IN INDICES*****

In [ ]:
import os
from tabulate import tabulate

def traverse_files(file1_path, file2_path):
    # Read the first file
    with open(file1_path, 'r') as f1:
        file1_data = f1.readlines()

    # Read the second file
    with open(file2_path, 'r') as f2:
        file2_data = f2.readlines()

    # Use a dictionary to store the second file's data by its first column values
    file2_dict = {}
    for line in file2_data:
        # Strip whitespace and split by commas
        parts = line.strip().split(',')
        if parts:  # Ensure line is not empty
            # Store the rest of the data by the first column
            file2_dict[parts[0]] = parts[1:]

    # Store matched results
    matched_results = []

    # Traverse through the first file and check for matches in the second file
    for line in file1_data:
        # Strip whitespace and split by commas
        parts = line.strip().split(',')
        if parts and parts[0] in file2_dict:
            # Prepare the matched result
            matched_results.append([parts[0]] + file2_dict[parts[0]])

    # Print the matched results in a tabular format
    if matched_results:
        headers = ['Index Code'] + [f'Column {i+1}' for i in range(len(matched_results[0]) - 1)]
        print(tabulate(matched_results, headers=headers,tablefmt='fancy_grid')) #tablefmt='psql'))
    else:
        print("No matches found!")

# Specify the paths to your files
file1_path = r"C:\\Users\manoj\Downloads\Major project data\Major pro source codes\DATASETS\indicesdf.csv"  # Replace with your actual file path
file2_path = r"C:\\Users\manoj\Downloads\Major project data\Major pro source codes\DATASETS\indicesstocks.csv"  # Replace with your actual file path

# Call the function
traverse_files(file1_path, file2_path)

*****GFS LOGIC FOR STOCKS*****

In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
from tabulate import tabulate

def append_row(df, row):
    """Appends a new row to a pandas DataFrame if the row is not empty."""
    if row.isnull().all():
        return df  # Do not append if the row is all NA
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14_and_BB(csvfilename):
    """Calculates RSI (14 period) and Bollinger Bands (20 period, 2 std.dev) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty or 'Close' not in df.columns:
                return 0.00, 0.00, 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                bb = ta.bbands(df['Close'], length=20)
                if bb is None or df['rsi14'] is None:
                    return 0.00, 0.00, 0.00, 0.00
                df['lowerband'] = bb['BBL_20_2.0']
                df['middleband'] = bb['BBM_20_2.0']
                if pd.isna(df['rsi14'].iloc[-1]) or pd.isna(df['lowerband'].iloc[-1]) or pd.isna(df['middleband'].iloc[-1]):
                    return 0.00, 0.00, 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                    lowerband = df['lowerband'].iloc[-1].round(2)
                    middleband = df['middleband'].iloc[-1].round(2)
                    return rsival, ltp, lowerband, middleband
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00, 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00, 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI, Bollinger Bands, and other metrics for daily, weekly, and monthly data."""
    # Replace periods with underscores in the symbol for file naming
    symbol_with_underscore = symbol.replace('.', '_')

    # Construct the file paths using the modified symbol
    daylocationstr = f'DATASETS/Daily_data/{symbol_with_underscore}.csv'
    weeklocationstr = f'DATASETS/Weekly_data/{symbol_with_underscore}.csv'
    monthlocationstr = f'DATASETS/Monthly_data/{symbol_with_underscore}.csv'

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp, daylowerband, daymiddleband = getRSI14_and_BB(daylocationstr)
    weekrsi14, wltp, weeklowerband, weekmiddleband = getRSI14_and_BB(weeklocationstr)
    monthrsi14, mltp, monthlowerband, monthmiddleband = getRSI14_and_BB(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14,
        'dltp': dltp,
        'daylowerband': daylowerband,
        'daymiddleband': daymiddleband,
        'weeklowerband': weeklowerband,
        'weekmiddleband': weekmiddleband,
        'monthlowerband': monthlowerband,
        'monthmiddleband': monthmiddleband
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14', 'dltp', 'daylowerband', 'daymiddleband', 'weeklowerband', 'weekmiddleband', 'monthlowerband', 'monthmiddleband'])

    fname = f'DATASETS/{scripttype}.csv'
    csvfilename = f'GFS_{scripttype}.csv'
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.replace("\n", "")
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

def read_indicesstocks(csvfilename):
    """Reads the indicesstocks CSV file and returns a dictionary of indices and their stocks."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename, header=None, on_bad_lines='skip')  # Use 'on_bad_lines' for newer versions
            indices_dict = {}
            for index, row in df.iterrows():
                index_code = row[0].strip()  # Get the index code
                stocks = row[1:].dropna().tolist()  # Get the stocks, drop NaN values
                indices_dict[index_code] = stocks
            return indices_dict
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return {}
    else:
        print(f"File does not exist: {csvfilename}")
        return {}

# Main execution
# Main execution
indicesdf_path = 'DATASETS/indicesdf.csv'
indicesstocks_path = r'C:\Users\manoj\Downloads\Major project data\Major pro source codes\DATASETS\indicesstocks.csv'
filtered_indices_path = 'DATASETS/filtered_indices.csv'  # Path for the new CSV file

# Generate GFS report
df3 = generateGFS('indicesdf')

# Filter based on criteria
df4 = df3.loc[
    (df3['monthrsi14'] >= 60.00) &
    (df3['weekrsi14'] >= 60.00) &
    df3['dayrsi14'].between(30, 70) &
    (df3['dltp'] > df3['daylowerband']) &
    (df3['dltp'] < df3['daymiddleband'])
]

# Extract only the indexcode for the filtered indices
filtered_indexcodes = df4[['indexcode']]

# Save the filtered indexcodes to a new CSV file, overwriting any existing file
filtered_indexcodes.to_csv(filtered_indices_path, index=False)

# Check if any indices qualified
if filtered_indexcodes.empty:
    print("\033[1mNO STOCKS QUALIFY THE GFS CRITERIA\033[0m")
else:
    # Read indices from indicesstocks
    indicesstocks = read_indicesstocks(indicesstocks_path)

    # Read filtered indices
    filtered_indices = filtered_indexcodes['indexcode'].tolist()

    # Compare and run GFS for matched indices
for index in filtered_indices:
    if index in indicesstocks:
        print(f"Running GFS for matched index: {index}")
        stocks = indicesstocks[index]  # Get the list of stocks for the matched index 
        
        # Traverse through the stocks starting from the next position
        # Inside your loop where you print the matched_row
for stock in stocks[1:]:  # Start from the second stock (index + 1)
    if stock:  # Check if stock is not empty
        print(f"Running GFS for stock: {stock}")
        try:
            # Call the GFS function for each stock
            matched_row = dayweekmonth_datasets(stock, stock)  # Using stock as both symbol and symbolname
            
            # Use tabulate to print the row with a specific format
            print(tabulate([matched_row], headers='keys', tablefmt='fancy_grid', showindex=False))
        except Exception as e:
            print(f"Error processing stock {stock}: {e}")
    else:
        print(f"Skipping empty stock name in index {index}.")

In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63


In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
import os

def append_row(df, row):
    """Appends a new row to a pandas DataFrame."""
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def getRSI14(csvfilename):
    """Calculates the RSI (14 period) for a given CSV file."""
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty:
                return 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                if pd.isna(df['rsi14'].iloc[-1]):
                    return 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                return rsival, ltp
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    """Calculates RSI for daily, weekly, and monthly data."""
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    # Adjust the paths to point to the DATASETS folder
    daylocationstr = f'../DATASETS/Daily_data/{symbol}.csv'  # Adjust path as needed
    weeklocationstr = f'../DATASETS/Weekly_data/{symbol}.csv'  # Adjust path as needed
    monthlocationstr = f'../DATASETS/Monthly_data/{symbol}.csv'  # Adjust path as needed

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp = getRSI14(daylocationstr)
    weekrsi14, wltp = getRSI14(weeklocationstr)
    monthrsi14, mltp = getRSI14(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14
    })
    return new_row

def generateGFS(scripttype):
    """Generates the GFS report based on the provided scripttype."""
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14'])

    # Adjust the path to point to the DATASETS folder
    fname = f'../DATASETS/{scripttype}'  # Adjust path as needed
    csvfilename = f'GFS_{scripttype}'  # Add .csv extension here
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.strip()  # Remove any leading/trailing whitespace
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

# Call the function without the .csv extension
df3 = generateGFS('indicesdf.csv')  # Call with just the base name
df3

C:\Users\manoj\AppData\Local\Temp\ipykernel_58056\2922273847.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)


,entrydate,indexcode,indexname,dayrsi14,weekrsi14,monthrsi14
0,12/01/2025,^NSEI,NIFTY 50\n,51.51,50.15,66.24
1,12/01/2025,^CNXAUTO,NIFTY AUTO\n,57.06,49.37,84.84
2,12/01/2025,^NSEBANK,NIFTY BANK\n,45.84,51.24,63.59
3,12/01/2025,^CNXFMCG,NIFTY FMCG\n,55.59,46.11,84.78
4,12/01/2025,^CNXIT,NIFTY IT\n,52.36,60.73,69.02
5,12/01/2025,^CNXMETAL,NIFTY METAL\n,35.38,42.76,71.94
6,12/01/2025,^CNXPHARMA,NIFTY PHARMA\n,0.00,0.00,0.00
7,12/01/2025,^CNXREALTY,NIFTY REALTY\n,42.86,51.89,84.86
8,12/01/2025,^CNXMEDIA,NIFTY MEDIA\n,31.89,40.62,46.95
9,12/01/2025,^CNXCONSUM,NIFTY CONSUMER GOODS,58.68,52.79,84.63
